In [ ]:
import numpy as np

# Initial parameters for components
pi1, mu1, sigma1_squared = 0.5, 2.0000137, 0.66674
pi2, mu2, sigma2_squared = 0.5, 6.99999, 0.66674

# Storage for data points and responsibilities
x_values = []
responsibilities = []

def gaussian_pdf(x, mu, sigma_squared):
    """Calculate the Gaussian PDF."""
    return (1 / np.sqrt(2 * np.pi * sigma_squared)) * np.exp(-((x - mu) ** 2) / (2 * sigma_squared))

def calculate_responsibilities(x, pi1, mu1, sigma1_squared, pi2, mu2, sigma2_squared):
    """Calculate responsibilities γ_i1 and γ_i2 for a data point x and print PDFs."""
    p_x_given_c1 = gaussian_pdf(x, mu1, sigma1_squared)
    p_x_given_c2 = gaussian_pdf(x, mu2, sigma2_squared)
    
    total = pi1 * p_x_given_c1 + pi2 * p_x_given_c2
    gamma_i1 = pi1 * p_x_given_c1 / total
    gamma_i2 = pi2 * p_x_given_c2 / total
    
    # Store x and its responsibilities
    x_values.append(x)
    responsibilities.append((gamma_i1, gamma_i2))
    
    # Print PDF values and responsibilities
    print(f"x = {x}")
    print(f"PDF for component 1: {p_x_given_c1}")
    print(f"PDF for component 2: {p_x_given_c2}")
    print(f"Responsibility γ_i1: {gamma_i1}, γ_i2: {gamma_i2}")
    
    return gamma_i1, gamma_i2

def maximization():
    """Perform maximization step to update model parameters and print updated values."""
    global pi1, mu1, sigma1_squared, pi2, mu2, sigma2_squared
    
    N = len(x_values)
    # Sum of responsibilities for each component
    sum_gamma1 = sum(g[0] for g in responsibilities)
    sum_gamma2 = sum(g[1] for g in responsibilities)
    
    # Update mixing coefficients
    pi1 = sum_gamma1 / N
    pi2 = sum_gamma2 / N
    
    # Update means
    mu1 = sum(g[0] * x for g, x in zip(responsibilities, x_values)) / sum_gamma1
    mu2 = sum(g[1] * x for g, x in zip(responsibilities, x_values)) / sum_gamma2
    
    # Update variances
    sigma1_squared = sum(g[0] * (x - mu1) ** 2 for g, x in zip(responsibilities, x_values)) / sum_gamma1
    sigma2_squared = sum(g[1] * (x - mu2) ** 2 for g, x in zip(responsibilities, x_values)) / sum_gamma2
    
    # Print updated parameters
    print("\nUpdated parameters after maximization:")
    print(f"pi1 = {pi1}, mu1 = {mu1}, sigma1_squared = {sigma1_squared}")
    print(f"pi2 = {pi2}, mu2 = {mu2}, sigma2_squared = {sigma2_squared}")

# Example usage of expectation and maximization
x_samples = [1,2,3,6,7,8]  # Example data points

# Expectation step for each data point
for x in x_samples:
    calculate_responsibilities(x, pi1, mu1, sigma1_squared, pi2, mu2, sigma2_squared)

# Maximization step
maximization()


In [ ]:
# Set parameters for a 3-component GMM
n_components = 3
n_features = iris_scaler.shape[1]
n_samples = iris_scaler.shape[0]

# Randomly initialize means, covariances, and mixing coefficients
np.random.seed(0)
means = np.random.rand(n_components, n_features)
covariances = np.array([np.eye(n_features) for _ in range(n_components)])
mixing_coeffs = np.full(n_components, 1 / n_components)  # Equal mixing coefficients



def e_step(X, means, covariances, mixing_coeffs):
    responsibilities = np.zeros((n_samples, n_components))

    for i in range(n_components):
        # Calculate the probability density for each Gaussian component
        pdf = multivariate_normal.pdf(X, mean=means[i], cov=covariances[i])
        responsibilities[:, i] = mixing_coeffs[i] * pdf

    # Normalize to get probabilities (responsibilities should sum to 1 across components)
    responsibilities /= responsibilities.sum(axis=1, keepdims=True)
    return responsibilities


In [ ]:
def m_step(X, responsibilities):
    # Update mixing coefficients
    Nk = responsibilities.sum(axis=0)
    mixing_coeffs = Nk / n_samples
    
    # Update means
    means = np.dot(responsibilities.T, X) / Nk[:, np.newaxis]

    # Update covariances
    covariances = []
    for i in range(n_components):
        X_centered = X - means[i]
        cov = (responsibilities[:, i, np.newaxis] * X_centered).T @ X_centered / Nk[i]
        covariances.append(cov)
    
    return means, np.array(covariances), mixing_coeffs


In [ ]:
max_iters = 100
tolerance = 1e-4  # Convergence tolerance
log_likelihoods = []

for iteration in range(max_iters):
    # E-step
    responsibilities = e_step(iris_scaler, means, covariances, mixing_coeffs)
    
    # M-step
    means, covariances, mixing_coeffs = m_step(iris_scaler, responsibilities)
    
    # Log likelihood (for convergence check)
    log_likelihood = np.sum(np.log(responsibilities.sum(axis=1)))
    log_likelihoods.append(log_likelihood)

    # Check for convergence
    if iteration > 0 and abs(log_likelihood - log_likelihoods[-2]) < tolerance:
        print(f"Converged at iteration {iteration}")
        break


In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
print("Final Means:\n", means)
print("Final Covariances:\n", covariances)
print("Final Mixing Coefficients:\n", mixing_coeffs)

# Reduce data to two dimensions for visualization
pca = PCA(n_components=2)
X_pca = pca.fit_transform(iris_scaler)

# Assign each point to the component with the highest responsibility
cluster_assignments = np.argmax(responsibilities, axis=1)

# Plot the data points and color by cluster assignment
plt.figure(figsize=(8, 6))
for i in range(n_components):
    plt.scatter(X_pca[cluster_assignments == i, 0], X_pca[cluster_assignments == i, 1], label=f'Cluster {i+1}')
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.legend()
plt.title("GMM Clustering on Iris Dataset (PCA-reduced)")
plt.show()
